"""

Constraints:
Each clothing item is given to exactly one child
Clothing size must match the chosen child’s size
Each child receives at least one summer item and one winter item


Objective:
◦ Let R = the sum of the retail prices for all of the clothing
◦ Let N = the number of children
◦ R/N = the fair share for each child
◦ Let Ci = the sum of the clothing prices for child i
◦ | R/N - Ci | = the amount over/under that child i received
◦ Your project code must compute a distribution that will provide the absolute minimum value for
D, where D is:


Signed Academic Integrity statement must be submitted


Your goal is to distribute them as fairly as possible, while ensuring that the clothes fit and that
each child gets at least one summer item and one winter item.
"""

In [43]:
import pandas as pd
def print_all_paths_helper(matrix_df, i, j, m, n, path, pi, pj, D_values, matrix_df_indices, matrix_df_cols,
                          matrix_df_i, matrix_df_j, r_by_n_ratio):
    if pi == m:
        return 
    if pj == n - 1:
        # reached the right end of column
        # print and return
        if matrix_df.loc[matrix_df_indices[pi],matrix_df_cols[pj]] > 0:
            path[i][j] = matrix_df.loc[matrix_df_indices[pi],matrix_df_cols[pj]]
            # TODO: Replace this with whitelist
            for idx in range(matrix_df_i):
                if idx == i:
                    continue
                path[idx][j] = None
        local_minimum = 0
        for row_idx in range(matrix_df_i):
            ci_this_child = 0
            for col_idx in range(matrix_df_j):
                if path[row_idx][col_idx] is not None:
                    ci_this_child += path[row_idx][col_idx]
            local_minimum += abs(r_by_n_ratio - ci_this_child)
        #print(local_minimum)
        D_values.append(local_minimum)
        #for row in path:
         #   print(row)
                
        #print("\n\n")
        # whitelist this j
        return
    # add them if only eligible
    # add current cell
    if matrix_df.loc[matrix_df_indices[pi],matrix_df_cols[pj]] > 0:
        path[i][j] = matrix_df.loc[matrix_df_indices[pi],matrix_df_cols[pj]]
        for idx in range(matrix_df_i):
            if idx == i:
                continue
            path[idx][j] = None
    else:
        if i + 1 < m:
            print_all_paths_helper(matrix_df, i+1, j , matrix_df_i, matrix_df_j, path, i + 1, j, D_values,
                                  matrix_df_indices, matrix_df_cols, matrix_df_i, matrix_df_j, r_by_n_ratio)
        if i - 1 >= 0:
            print_all_paths_helper(matrix_df, i-1, j, matrix_df_i, matrix_df_j, path, i - 1, j, D_values,
                                  matrix_df_indices, matrix_df_cols, matrix_df_i, matrix_df_j, r_by_n_ratio)


            
    # print all paths that are possible moving to next column
    #for ix in range(i, m):
    print_all_paths_helper(matrix_df, i, j + 1, matrix_df_i, matrix_df_j, path, pi, j + 1, D_values,
                          matrix_df_indices, matrix_df_cols, matrix_df_i, matrix_df_j, r_by_n_ratio)
    #print_all_paths_helper(matrix_df, i+1, j + 1, matrix_df_i, matrix_df_j, path, i + 1, j + 1, whitelisted_j)
    

def print_all_paths(fnames):

    for fn in fnames:
        #print(fn)
        with open(f"example_input/{fn}") as f:
            flines = f.readlines()
            clothes_records  = []
            children_records = []
            for fline in flines:
                if 'Clothes' in fline or 'Children' in fline:
                    continue
                else:
                    if fline[0] == 'A':
                        cloth_record = fline.split("\t")
                        clothes_records.append({
                                                       'cloth_id': cloth_record[0],
                                                       'cloth_size': cloth_record[1],
                                                       'cloth_season': cloth_record[2],
                                                       'cloth_price': cloth_record[3].strip()
                                               }
                                              )
                    else:
                        children_record = fline.split("\t")
                        children_records.append(
                            {
                                'child_id': children_record[0],
                                'child_size': children_record[1].strip(),
                                'summer_clothes': [],
                                'winter_clothes': []
                            }
                        )
            N = len(children_records) # the number of children
            R = 0
            for cloth_record in clothes_records:
                R += float(cloth_record['cloth_price'])

            ## R/N = the fair share for each child
            r_by_n_ratio = R/N
            child_ids = [f"{each['child_id']}_{each['child_size']}" for each in children_records]
            summer_cloth_ids = [f'SU_{each["cloth_id"]}_{each["cloth_size"]}_{each["cloth_price"]}' for each in clothes_records if each['cloth_season']=='summer']
            winter_cloth_ids = [f'WI_{each["cloth_id"]}_{each["cloth_size"]}_{each["cloth_price"]}' for each in clothes_records if each['cloth_season']=='winter']

            summer_df = pd.DataFrame(index=child_ids, columns=summer_cloth_ids)
            winter_df = pd.DataFrame(index=child_ids, columns=winter_cloth_ids)

            summer_df_i, summer_df_j = summer_df.shape
            summer_df_cols = summer_df.columns.tolist()
            summer_df_indices = summer_df.index.tolist()


            for i in range(summer_df_i):
                for j in range(summer_df_j):        
                    cl_size = summer_df_cols[j].split("_")[2]
                    cl_price = summer_df_cols[j].split("_")[3]
                    cd_size = summer_df_indices[i].split("_")[1]
                    if cl_size == cd_size or cl_size == "ALL":
                        # assign this cloth to the child
                        summer_df.loc[summer_df_indices[i], summer_df_cols[j]] = cl_price

            #print(summer_df)
            winter_df_i, winter_df_j = winter_df.shape
            winter_df_cols = winter_df.columns.tolist()
            #print(winter_df_cols)
            winter_df_indices = winter_df.index.tolist()
            #print(winter_df_indices)
            for i in range(winter_df_i):
                for j in range(winter_df_j):
                    cl_size = winter_df_cols[j].split("_")[2]
                    cl_price = winter_df_cols[j].split("_")[3]
                    cd_size = winter_df_indices[i].split("_")[1]
                    if cl_size == cd_size or cl_size == "ALL":
                        # assign this cloth to the child
                        winter_df.loc[winter_df_indices[i], winter_df_cols[j]] = cl_price

            matrix_df = summer_df.join(winter_df)
            matrix_df = matrix_df.apply(pd.to_numeric)
            matrix_df.fillna(-1, inplace=True)

            matrix_df_i, matrix_df_j = matrix_df.shape
            matrix_df_indices = matrix_df.index.tolist()
            matrix_df_cols = matrix_df.columns.tolist()






            D_values = []
            path = [[None for j in range(matrix_df_j)] for i in range(matrix_df_i)]
            try: 
                print_all_paths_helper(matrix_df, 0, 0, matrix_df_i, matrix_df_j, path, 0, 0, D_values,
                                      matrix_df_indices, matrix_df_cols, matrix_df_i, matrix_df_j, r_by_n_ratio)
                print(f"\n####Final optimal global minimum of example {fn}: {min(D_values)}####\n")
            except RecursionError:
                #print(f"\n*****Skipping the file {fn} due to recursion error***\n")
                pass
            # TODO: Time it
            
        

In [44]:
import os
fnames = os.listdir("example_input/")

In [45]:
fnames[:5]

['ex1_2children_4clothes.txt',
 'ex10_3children_11clothes.txt',
 'ex4_2children_6clothes.txt',
 'ex28_20children_100clothes.txt',
 'ex2_2children_4clothes.txt']

In [46]:
print_all_paths(fnames)



####Final optimal global minimum of example ex1_2children_4clothes.txt: 16.0####


####Final optimal global minimum of example ex4_2children_6clothes.txt: 9.0####


####Final optimal global minimum of example ex2_2children_4clothes.txt: 26.0####


####Final optimal global minimum of example ex3_2children_4clothes.txt: 17.0####


####Final optimal global minimum of example ex11_3children_12clothes.txt: 170.0####


####Final optimal global minimum of example ex6_2children_6clothes.txt: 49.0####


####Final optimal global minimum of example ex5_2children_6clothes.txt: 15.0####


####Final optimal global minimum of example ex7_2children_7clothes.txt: 10.0####


####Final optimal global minimum of example ex8_2children_10clothes.txt: 9.0####

